In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import pymc3 as pm
import matplotlib.pyplot as plt
import pystan

In [2]:
plt.style.use('seaborn-dark')

Dimensions: (region, medium) $\equiv (i,j)$

$Y_{i,j} = A_i + B_j + \varepsilon$

$\varepsilon \sim {\cal N}(0,\sigma_{i,j})$

In [3]:
class experiment():

    def __init__(self, num_regions, num_media):
        
        self.num_regions = num_regions
        self.num_media = num_media
        self.parameters = {
            'A': stats.uniform(loc=2, scale=3).rvs(size=num_regions),
            'B': stats.norm(loc=1, scale=5).rvs(size=num_media),
            'sigma': stats.pareto(b=1).rvs(size=(num_regions, num_media)),
        }
        
    def get_samples(self, num_samples):
        i = np.random.choice(self.num_regions, size=num_samples)
        j = np.random.choice(self.num_media, size=num_samples)
        A_s = self.parameters['A'][i]
        B_s = self.parameters['B'][j]
        sigma_s = self.parameters['sigma'][i,j]
        Y_s = A_s + B_s + sigma_s*np.random.normal(size=num_samples)
        return pd.DataFrame({
            'i':i,
            'j':j,
            'Y':Y_s
        })

In [4]:
num_regions = 4
num_media = 3
num_samples = 10000

exp = experiment(num_regions=num_regions, num_media=num_media)
df = exp.get_samples(num_samples=num_samples)
df.head()

,i,j,Y
0,1,2,8.974070
1,1,1,-0.767923
2,3,0,2.122252
3,2,2,0.363723
4,2,2,58.843530


In [5]:
stan_model = """
    data {
        int<lower=0> num_samples;
        int<lower=0> num_i;
        int<lower=0> num_j;
        int<lower=1> i[num_samples];
        int<lower=1> j[num_samples];
        int<lower=1> ij[num_samples];
        vector[num_samples] Y;
    }
    parameters {
        vector<lower=0>[num_i] A;
        vector[num_j] B;
        vector<lower=0>[num_i*num_j] sigma;
    }
    model {
        A ~ normal(0.0, 10.0);
        B ~ normal(0.0, 10.0);
        sigma ~ normal(0.0, 100.0);
        Y ~ normal(A[i] + B[j], sigma[ij]);
    }
"""
sm = pystan.StanModel(model_code=stan_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1b08000e74b1e399bff5e749dbeb8105 NOW.


In [ ]:
data = {
    'num_i': num_regions,
    'num_j': num_media,
    'num_samples': num_samples,
    'i': df['i'].values + 1,
    'j': df['j'].values + 1,
    'ij': (df['i']*num_media + df['j']) + 1,
    'Y': df['Y'].values
}
fit = sm.sampling(
    data=data,
    iter=1000,
    warmup=500,
    n_jobs=-1,
    chains=2
)

In [ ]:
fit.plot()

In [ ]:
plt.hist(fit.extract()['B'][:,0], bins=50)